<a href="https://colab.research.google.com/github/mmorari-cmyk/CURSO_SE.ALES_Y_SISTEMAS/blob/main/Taller2/ejercicio_2_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

In [34]:
%%writefile SLIT.py

# SECCIÓN 1: IMPORTACIÓN DE LIBRERÍAS

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import sympy as sym
import sympy.physics.control as control


# SECCIÓN 2: CONFIGURACIÓN DE LA PÁGINA

st.set_page_config(page_title="Simulación RLC 2º Orden", layout="wide")

# Sidebar con información del autor
st.sidebar.title("Presentado por:")
st.sidebar.write("Marlyn Nathalia Mora Riascos")


# SECCIÓN 3: TEXTO INTRODUCTORIO Y TEORÍA
print()
st.title("Caracterización de un SLIT de segundo orden (RLC serie/paralelo)según el factor de amortiguamiento")

st.write("""
Sistema lineal, continuo, invariante en el tiempo (SLIT) de segundo orden, modelado por circuitos RLC serie y paralelo con distintos factores de amortiguamiento (subamortiguado, crítico, sobreamortiguado e inestable).
""")

# Explicación teórica
st.write("""
En un sistema de segundo orden, el comportamiento de la respuesta en el tiempo está determinado por el factor de amortiguamiento ξ y por la posición de sus polos en el plano complejo (las raíces del denominador de la función de transferencia H(s)). Estos polos indican si la señal se acerca al valor final de forma rápida, lenta, con oscilaciones o sin ellas.

**Sistema sobreamortiguado (ξ > 1)**: sus polos son reales, negativos y distintos, lo que produce una respuesta que se acerca al valor final de la grafica de forma lenta y suave, sin oscilaciones ni sobreimpulso. El sistema está muy frenado: es muy estable, pero tarda más en llegar al valor deseado.

**Sistema subamortiguado (0 < ξ < 1)**: sus polos son complejos conjugados con parte real negativa. Esto hace que la respuesta sea rápida pero oscilatoria: la señal se pasa del valor final, luego baja, vuelve a subir y así, generando “onditas” que se van apagando con el tiempo. Es el comportamiento típico de un sistema que vibra antes de estabilizarse.

**Sistema con amortiguamiento crítico (ξ = 1)**: sus polos son reales, negativos e iguales (polo doble). En este caso la respuesta no oscila y alcanza el valor final en el menor tiempo posible sin sobrepasarse. Es el punto intermedio ideal entre reaccionar demasiado rápido (con oscilaciones) y ser demasiado lento.
""")



# Inicialización de SymPy para mostrar fórmulas de manera simbolica
sym.init_printing()

# Definición de variables simbólicas para las fórmulas
s = sym.symbols('s', complex=True)
R_sym, L_sym, C_sym = sym.symbols('R L C', positive=True, real=True)

# Visualización de fórmulas matemáticas con LaTeX
with st.expander("Formulación teórica (ξ, ωₙ, ω_d en función de R, L, C)"):
    st.markdown("**Circuito RLC serie (Salida: Voltaje en Capacitor)**")
    st.latex(r"\omega_n = \frac{1}{\sqrt{LC}}, \qquad"
             r"\xi = \frac{R}{2}\sqrt{\frac{C}{L}}, \qquad"
             r"\omega_d = \omega_n\sqrt{1-\xi^2}")

    st.markdown("**Circuito RLC paralelo (Salida: Voltaje o Corriente)**")
    st.latex(r"\omega_n = \frac{1}{\sqrt{LC}}, \qquad"
             r"\xi = \frac{1}{2R}\sqrt{\frac{L}{C}}, \qquad"
             r"\omega_d = \omega_n\sqrt{1-\xi^2}")


# SECCIÓN 4: FUNCIONES DE DISEÑO Y CÁLCULO


# Valor base de capacitancia para poder despejar L y R
CAPACITANCIA_BASE = 1.0  # 1 Faradio (Valor para facilitar simulación)

def disenar_rlc_serie(zeta, omega_n, C_valor=CAPACITANCIA_BASE):
    """
    Calcula R y L para un circuito Serie dado un factor de amortiguamiento (zeta)
    y frecuencia natural (omega_n).
    """
    # Despeje: omega_n = 1/sqrt(L*C)  ->  L = 1 / (omega_n^2 * C)
    if omega_n == 0: omega_n = 0.001 # Evitar división por cero
    L_calculada = 1 / (omega_n**2 * C_valor)

    # Despeje: zeta = (R/2)*sqrt(C/L) ->  R = 2 * zeta * sqrt(L/C)
    R_calculada = 2 * zeta * np.sqrt(L_calculada / C_valor)

    # Función de transferencia numérica para graficar (usando scipy.signal)
    # H(s) = omega_n^2 / (s^2 + 2*zeta*omega_n*s + omega_n^2)
    numerador = [omega_n**2]
    denominador = [1, 2*zeta*omega_n, omega_n**2]
    sistema_lti = signal.TransferFunction(numerador, denominador)

    return R_calculada, L_calculada, C_valor, sistema_lti

def disenar_rlc_paralelo(zeta, omega_n, C_valor=CAPACITANCIA_BASE):
    """
    Calcula R y L para un circuito Paralelo.
    """
    if omega_n == 0: omega_n = 0.001
    L_calculada = 1 / (omega_n**2 * C_valor)

    # Despeje Paralelo: zeta = (1/(2R))*sqrt(L/C) -> R = sqrt(L/C) / (2*zeta)
    # Nota: Si zeta es 0 o negativo muy cercano a 0, R tiende a infinito (circuito abierto)
    if abs(zeta) < 1e-9:
        R_calculada = 1e9
    else:
        R_calculada = np.sqrt(L_calculada / C_valor) / (2 * zeta)

    # H(s) normalizada de 2do orden
    numerador = [omega_n**2]
    denominador = [1, 2*zeta*omega_n, omega_n**2]
    sistema_lti = signal.TransferFunction(numerador, denominador)

    return R_calculada, L_calculada, C_valor, sistema_lti

def calcular_metricas_tiempo(t, y, valor_final=1.0):
    """
    Analiza la respuesta al escalón para encontrar:
    - Tiempo de levantamiento (Tr)
    - Máximo sobreimpulso (Mp)
    - Tiempo pico (Tp)
    - Tiempo de establecimiento (Ts)
    """
    # 1. Tiempo de levantamiento (10% al 90%)
    try:
        idx_10 = np.where(y >= 0.1 * valor_final)[0][0]
        idx_90 = np.where(y >= 0.9 * valor_final)[0][0]
        t_r = t[idx_90] - t[idx_10]
    except:
        t_r = np.nan

    # 2. Máximo sobreimpulso
    val_max = np.max(y)
    if val_max > valor_final:
        M_p = (val_max - valor_final) / valor_final * 100
        # 3. Tiempo pico (solo si hay sobreimpulso)
        idx_pico = np.argmax(y)
        t_p = t[idx_pico]
    else:
        M_p = 0.0
        t_p = np.nan

    # 4. Tiempo de establecimiento (Criterio del 2%)
    t_s = np.nan
    banda_error = 0.02 * valor_final
    # Recorremos el array hacia atrás para ver cuándo entra en la banda y no vuelve a salir
    for i in range(len(y)-1, 0, -1):
        if abs(y[i] - valor_final) > banda_error:
            t_s = t[i]
            break

    return t_r, M_p, t_p, t_s



# SECCIÓN 5: INTERFAZ DE USUARIO (SIDEBAR)

st.sidebar.header("Parámetros de Entrada")

tipo_circuito = st.sidebar.selectbox("Seleccione Tipo de Circuito", ["RLC Serie", "RLC Paralelo"])
tipo_respuesta = st.sidebar.selectbox("Tipo de Respuesta Deseada",
    ["Subamortiguado", "Sobreamortiguado", "Amortiguamiento crítico", "Inestable"])

omega_n = st.sidebar.slider("Frecuencia Natural (ωn) [rad/s]", 0.5, 50.0, 10.0, step=0.5)

# Lógica condicional para el slider de amortiguamiento (zeta)
if tipo_respuesta == "Subamortiguado":
    coef_amort = st.sidebar.slider("Factor de amortiguamiento ξ (0 < ξ < 1)", 0.05, 0.95, 0.3, 0.05)
elif tipo_respuesta == "Sobreamortiguado":
    coef_amort = st.sidebar.slider("Factor de amortiguamiento ξ (ξ > 1)", 1.05, 5.0, 2.0, 0.05)
elif tipo_respuesta == "Amortiguamiento crítico":
    coef_amort = 1.0
    st.sidebar.info("Factor ξ fijado en 1.0")
else: # Inestable
    coef_amort = st.sidebar.slider("Factor de amortiguamiento ξ (ξ < 0)", -1.0, -0.05, -0.3, 0.05)


# SECCIÓN 6: EJECUCIÓN DE CÁLCULOS


# Calcular valores R, L, C y obtener el sistema
if tipo_circuito == "RLC Serie":
    R_est, L_est, C_est, sistema = disenar_rlc_serie(coef_amort, omega_n)
else:
    R_est, L_est, C_est, sistema = disenar_rlc_paralelo(coef_amort, omega_n)

# Calcular Frecuencia amortiguada (wd) solo si es subamortiguado
wd_calc = omega_n * np.sqrt(1 - coef_amort**2) if (0 < coef_amort < 1) else 0

# Mostrar parámetros estimados
st.subheader("Parámetros Estimados del Circuito")
col1, col2, col3 = st.columns(3)
col1.metric("Resistencia (R)", f"{abs(R_est):.4g} Ω")
col2.metric("Inductancia (L)", f"{L_est:.4g} H")
col3.metric("Capacitancia (C)", f"{C_est:.4g} F")

col4, col5, col6 = st.columns(3)
col4.metric("Frecuencia Natural (ωn)", f"{omega_n:.4g} rad/s")
col5.metric("Factor Amortiguamiento (ξ)", f"{coef_amort:.4g}")
col6.metric("Frecuencia Amortiguada (ωd)", f"{wd_calc:.4g} rad/s" if 0 < coef_amort < 1 else "No aplica")



# SECCIÓN 7: GENERACIÓN DE GRÁFICAS Y RESULTADOS (CON API DE CONTROL SYMPY)



# Paso previo: Construir la Función de Transferencia Simbólica con la API de control de SymPy trabaja con objetos simbólicos, no con arrays.

s = sym.Symbol('s') # Variable compleja s

# Definimos la función de transferencia estándar de 2do orden:
# H(s) = wn^2 / (s^2 + 2*zeta*wn*s + wn^2)
numerador_sym = omega_n**2
denominador_sym = s**2 + 2 * coef_amort * omega_n * s + omega_n**2
H_RLC = control.TransferFunction(numerador_sym, denominador_sym, s)

# Definimos un límite de tiempo visual dinámico para las gráficas
t_limite = 15.0 / (omega_n * abs(coef_amort)) if (omega_n > 0 and abs(coef_amort) > 0.1) else 10.0

st.markdown("---")
st.success("Gráficas generadas utilizando **SymPy Physics Control API**")

# 1. Diagrama de Bode

st.subheader("1. Diagrama de Bode (Frecuencia)")
# bode_plot genera automáticamente magnitud y fase
control.bode_plot(H_RLC, show=False)
fig_bode = plt.gcf()      # Obtener la figura actual de matplotlib generada por sympy
st.pyplot(fig_bode)       # Mostrar en Streamlit
plt.clf()                 # Limpiar la figura para no mezclar con la siguiente

# 2. Mapa de Polos y Ceros

st.subheader("2. Mapa de Polos y Ceros")
control.pole_zero_plot(H_RLC, show=False)
fig_pz = plt.gcf()
st.pyplot(fig_pz)
plt.clf()

# 3. Respuestas en el Tiempo

st.subheader("3. Respuestas en el Dominio del Tiempo")

# Organizar gráficas en columnas para mejor visualización

col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("**Respuesta al Impulso**")
    control.impulse_response_plot(H_RLC, upper_limit=t_limite, show=False)
    st.pyplot(plt.gcf())
    plt.clf()

with col2:
    st.markdown("**Respuesta al Escalón**")
    control.step_response_plot(H_RLC, upper_limit=t_limite, show=False)
    st.pyplot(plt.gcf())
    plt.clf()

with col3:
    st.markdown("**Respuesta a la Rampa**")
    control.ramp_response_plot(H_RLC, upper_limit=t_limite, show=False)
    st.pyplot(plt.gcf())
    plt.clf()


# SECCIÓN 8: MÉTRICAS DEL DESEMPEÑO

st.subheader("4. Métricas de Desempeño (Respuesta al Escalón)")

if tipo_respuesta == "Inestable":
    st.warning("El sistema es inestable. Las métricas de tiempo no están definidas.")
else:
    # 1. Crear función de transferencia numérica temporal
    # H(s) = wn^2 / (s^2 + 2*zeta*wn*s + wn^2)
    num_calc = [omega_n**2]
    den_calc = [1, 2 * coef_amort * omega_n, omega_n**2]
    sistema_calculo = signal.TransferFunction(num_calc, den_calc)

    # 2. Generar los datos numéricos (Usamos el mismo t_limite de la Sección 7)
    # t_limite viene de la sección anterior, aseguramos que exista.
    if 't_limite' not in locals(): t_limite = 10.0

    t_simulacion = np.linspace(0, t_limite, 2000)
    t_step, y_step = signal.step(sistema_calculo, T=t_simulacion)

    # 3. Llamar a la función de cálculo de métricas
    tr, mp, tp, ts = calcular_metricas_tiempo(t_step, y_step)

    # 4. Mostrar resultados con st.metric para mejor visualización
    col_met1, col_met2 = st.columns(2)

    # Formateo de strings para manejar valores nulos (NaN)
    str_tr = f"{tr:.4f} s" if not np.isnan(tr) else "No definido"
    str_mp = f"{mp:.2f} %"
    str_tp = f"{tp:.4f} s" if not np.isnan(tp) else "No aplica"
    str_ts = f"{ts:.4f} s" if not np.isnan(ts) else "No converge"

    col_met1.metric("Tiempo de Levantamiento (Tr)", str_tr)
    col_met1.metric("Máximo Sobreimpulso (Mp)", str_mp)

    col_met2.metric("Tiempo Pico (Tp)", str_tp)
    col_met2.metric("Tiempo de Establecimiento (Ts 2%)", str_ts)



st.markdown("---")
st.write("""
Este simulador permite entender cómo se comportan circuitos RLC de segundo orden (serie y paralelo) cuando se cambian sus parámetros.

Al elegir el tipo de circuito, el tipo de respuesta (subamortiguada, sobreamortiguada, crítica o inestable), el factor de amortiguamiento y la frecuencia natural, el sistema calcula valores de R, L y C que cumplen esas condiciones.

A partir de esto, se muestra:
- Cómo responde el circuito en el tiempo (impulso, escalón y rampa).
- Cómo se distribuyen sus polos y ceros en el plano complejo.
- Cómo se comporta en frecuencia (diagrama de Bode).
- Tiempos importantes como el tiempo de levantamiento, el máximo sobreimpulso, el tiempo del máximo sobreimpulso y el tiempo de establecimiento.

Esta herramienta ayuda a visualizar cómo los valores de R, L y C influyen en la rapidez, la estabilidad y las oscilaciones del sistema, sin necesidad de armar físicamente el circuito.
""")

Overwriting SLIT.py


In [35]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run SLIT.py &>/content/logs.txt & #Cambiar  ...py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-26 20:40:42--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-26 20:40:43--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-26T21%3A20%3A40Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-